In [1]:
!pip install tensorflow -q
!pip install -U "seaborn" --quiet

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# --- Setup, Model Definition, Helper/Physics Functions ---
tf.keras.backend.set_floatx('float64')
DEVICE = "/GPU:0" if tf.config.list_physical_devices('GPU') else "/CPU:0"
print(f"Using device: {DEVICE}")

ARG_CLIP_MIN = tf.constant(-10.0, dtype=tf.float64)
ARG_CLIP_MAX = tf.constant(10.0, dtype=tf.float64)

@tf.function
def smooth_relu(x, beta=20.0):
    """A smooth, differentiable approximation of the ReLU function."""
    return tf.nn.softplus(beta * x) / beta

Using device: /GPU:0


I0000 00:00:1755090042.372533   14578 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1238 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
class StrainEnergyANN_Layered_TF(tf.keras.Model):
    def __init__(self):
        super().__init__(name="StrainEnergyModel")
        # --- MODIFICATION: REMOVE b1 FROM TRAINABLE PARAMETERS ---
        # b1 will be calculated from the other parameters to enforce the constraint.
        # self.raw_log_b1 = self.add_weight(...) # This line is removed.

        # k & i params (exponents) - All other parameters remain trainable
        self.raw_log_k1=self.add_weight(name="raw_log_k1",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k2=self.add_weight(name="raw_log_k2",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k3=self.add_weight(name="raw_log_k3",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k4=self.add_weight(name="raw_log_k4",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k5=self.add_weight(name="raw_log_k5",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k6=self.add_weight(name="raw_log_k6",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k7=self.add_weight(name="raw_log_k7",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k8=self.add_weight(name="raw_log_k8",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k9=self.add_weight(name="raw_log_k9",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k10=self.add_weight(name="raw_log_k10",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k11=self.add_weight(name="raw_log_k11",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k12=self.add_weight(name="raw_log_k12",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k13=self.add_weight(name="raw_log_k13",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k14=self.add_weight(name="raw_log_k14",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k15=self.add_weight(name="raw_log_k15",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_k16=self.add_weight(name="raw_log_k16",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i1=self.add_weight(name="raw_log_i1",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i2=self.add_weight(name="raw_log_i2",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i3=self.add_weight(name="raw_log_i3",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i4=self.add_weight(name="raw_log_i4",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i5=self.add_weight(name="raw_log_i5",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i6=self.add_weight(name="raw_log_i6",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i7=self.add_weight(name="raw_log_i7",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_i8=self.add_weight(name="raw_log_i8",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.RandomNormal(mean=0.0,stddev=0.01),trainable=True)
        self.raw_log_a1=self.add_weight(name="raw_log_a1",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a2=self.add_weight(name="raw_log_a2",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a3=self.add_weight(name="raw_log_a3",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a4=self.add_weight(name="raw_log_a4",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a5=self.add_weight(name="raw_log_a5",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a6=self.add_weight(name="raw_log_a6",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a7=self.add_weight(name="raw_log_a7",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a8=self.add_weight(name="raw_log_a8",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b2=self.add_weight(name="raw_log_b2",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.01)),trainable=True)
        self.raw_log_b3=self.add_weight(name="raw_log_b3",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b4=self.add_weight(name="raw_log_b4",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b5=self.add_weight(name="raw_log_b5",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b6=self.add_weight(name="raw_log_b6",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b7=self.add_weight(name="raw_log_b7",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_b8=self.add_weight(name="raw_log_b8",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.1)),trainable=True)
        self.raw_log_a3_prime=self.add_weight(name="raw_log_a3_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_a4_prime=self.add_weight(name="raw_log_a4_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_a5_prime=self.add_weight(name="raw_log_a5_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_a6_prime=self.add_weight(name="raw_log_a6_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_a7_prime=self.add_weight(name="raw_log_a7_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_a8_prime=self.add_weight(name="raw_log_a8_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_b3_prime=self.add_weight(name="raw_log_b3_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_b4_prime=self.add_weight(name="raw_log_b4_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_b5_prime=self.add_weight(name="raw_log_b5_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_b6_prime=self.add_weight(name="raw_log_b6_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_b7_prime=self.add_weight(name="raw_log_b7_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)
        self.raw_log_b8_prime=self.add_weight(name="raw_log_b8_prime",shape=(),dtype=tf.float64,initializer=tf.keras.initializers.Constant(tf.math.log(0.2)),trainable=True)

        self.three=tf.constant(3.0,dtype=tf.float64)
        self.one=tf.constant(1.0,dtype=tf.float64)
        self.pow_base_epsilon=tf.constant(1e-8,dtype=tf.float64)

    def _term_power_law(self, I, k, i, c, ref_val):
        arg = smooth_relu(tf.pow(I, k) - tf.pow(ref_val, k))
        return c * tf.pow(arg + self.pow_base_epsilon, i)
    def _term_exponential(self, I, k, i, ic, oc, ref_val):
        arg = smooth_relu(tf.pow(I, k) - tf.pow(ref_val, k))
        return oc * (tf.exp(tf.clip_by_value(ic * tf.pow(arg + self.pow_base_epsilon, i), ARG_CLIP_MIN, ARG_CLIP_MAX)) - 1.0)
    def _term_cosh_minus_one_with_i(self, I, k, i, ic, oc, ref_val):
        arg = smooth_relu(tf.pow(I, k) - tf.pow(ref_val, k))
        return oc * (tf.cosh(tf.clip_by_value(ic * tf.pow(arg + self.pow_base_epsilon, i), ARG_CLIP_MIN, ARG_CLIP_MAX)) - 1.0)
    def _term_sinh_with_i(self, I, k, i, ic, oc, ref_val):
        arg = smooth_relu(tf.pow(I, k) - tf.pow(ref_val, k))
        return oc * tf.sinh(tf.clip_by_value(ic * tf.pow(arg + self.pow_base_epsilon, i), ARG_CLIP_MIN, ARG_CLIP_MAX))
    def _term_identity_scaled(self, I, k, c, ref_val):
        return c * (tf.pow(I, k) - tf.pow(ref_val, k))
    def _term_exponential_no_i(self, I, k, ic, oc, ref_val):
        return oc * (tf.exp(tf.clip_by_value(ic * (tf.pow(I, k) - tf.pow(ref_val, k)), ARG_CLIP_MIN, ARG_CLIP_MAX)) - 1.0)
    def _term_cosh_minus_one(self, I, k, ic, oc, ref_val):
        return oc * (tf.cosh(tf.clip_by_value(ic * (tf.pow(I, k) - tf.pow(ref_val, k)), ARG_CLIP_MIN, ARG_CLIP_MAX)) - 1.0)
    def _term_sinh(self, I, k, ic, oc, ref_val):
        return oc * tf.sinh(tf.clip_by_value(ic * (tf.pow(I, k) - tf.pow(ref_val, k)), ARG_CLIP_MIN, ARG_CLIP_MAX))

    def call(self, I1, I2, I4, I6):
        k1=1.0+tf.exp(self.raw_log_k1);k2=1.5+tf.exp(self.raw_log_k2);k3=1.0+tf.exp(self.raw_log_k3);k4=1.5+tf.exp(self.raw_log_k4);k5=1.0+tf.exp(self.raw_log_k5);k6=1.5+tf.exp(self.raw_log_k6);k7=1.0+tf.exp(self.raw_log_k7);k8=1.5+tf.exp(self.raw_log_k8);k9=1.0+tf.exp(self.raw_log_k9);k10=1.5+tf.exp(self.raw_log_k10);k11=1.0+tf.exp(self.raw_log_k11);k12=1.5+tf.exp(self.raw_log_k12);k13=1.0+tf.exp(self.raw_log_k13);k14=1.5+tf.exp(self.raw_log_k14);k15=1.0+tf.exp(self.raw_log_k15);k16=1.5+tf.exp(self.raw_log_k16)
        i1=1.0+tf.exp(self.raw_log_i1);i2=1.0+tf.exp(self.raw_log_i2);i3=1.0+tf.exp(self.raw_log_i3);i4=1.0+tf.exp(self.raw_log_i4);i5=1.0+tf.exp(self.raw_log_i5);i6=1.0+tf.exp(self.raw_log_i6);i7=1.0+tf.exp(self.raw_log_i7);i8=1.0+tf.exp(self.raw_log_i8)
        a1=tf.exp(self.raw_log_a1);a2=tf.exp(self.raw_log_a2);a3=tf.exp(self.raw_log_a3);a4=tf.exp(self.raw_log_a4);a5=tf.exp(self.raw_log_a5);a6=tf.exp(self.raw_log_a6);a7=tf.exp(self.raw_log_a7);a8=tf.exp(self.raw_log_a8)
        a3_prime=tf.exp(self.raw_log_a3_prime);a4_prime=tf.exp(self.raw_log_a4_prime);a5_prime=tf.exp(self.raw_log_a5_prime);a6_prime=tf.exp(self.raw_log_a6_prime);a7_prime=tf.exp(self.raw_log_a7_prime);a8_prime=tf.exp(self.raw_log_a8_prime)

        b2=tf.exp(self.raw_log_b2);b3=tf.exp(self.raw_log_b3);b4=tf.exp(self.raw_log_b4);b5=tf.exp(self.raw_log_b5);b6=tf.exp(self.raw_log_b6);b7=tf.exp(self.raw_log_b7);b8=tf.exp(self.raw_log_b8)
        b3_prime=tf.exp(self.raw_log_b3_prime);b4_prime=tf.exp(self.raw_log_b4_prime);b5_prime=tf.exp(self.raw_log_b5_prime);b6_prime=tf.exp(self.raw_log_b6_prime);b7_prime=tf.exp(self.raw_log_b7_prime);b8_prime=tf.exp(self.raw_log_b8_prime)

        # --- MODIFICATION: ENFORCE CONSTRAINT VIA REPARAMETERIZATION ---
        rhs = (b2 * k10) + (b4 * b4_prime * k12) + (b8 * b8_prime * k16)
        lhs_sub = (b3 * b3_prime * k11) + (b7 * b7_prime * k15)
        # k9 is guaranteed to be > 1.0, so division is safe. Add a small epsilon for absolute stability.
        b1 = (rhs - lhs_sub) / (k9 + 1e-8)

        W = tf.zeros_like(I1,dtype=tf.float64)
        W += self._term_power_law(I1,k1,i1,a1,self.three); W += self._term_power_law(I2,k2,i2,a2,self.three)
        W += self._term_exponential(I1,k3,i3,a3_prime,a3,self.three); W += self._term_exponential(I2,k4,i4,a4_prime,a4,self.three)
        W += self._term_cosh_minus_one_with_i(I1,k5,i5,a5_prime,a5,self.three); W += self._term_cosh_minus_one_with_i(I2,k6,i6,a6_prime,a6,self.three)
        W += self._term_sinh_with_i(I1,k7,i7,a7_prime,a7,self.three); W += self._term_sinh_with_i(I2,k8,i8,a8_prime,a8,self.three)
        # Use the computed b1 here
        W += self._term_identity_scaled(I4,k9,b1,self.one); W += self._term_identity_scaled(I6,k10,b2,self.one)
        W += self._term_exponential_no_i(I4,k11,b3_prime,b3,self.one); W += self._term_exponential_no_i(I6,k12,b4_prime,b4,self.one)
        W += self._term_cosh_minus_one(I4,k13,b5_prime,b5,self.one); W += self._term_cosh_minus_one(I6,k14,b6_prime,b6,self.one)
        W += self._term_sinh(I4,k15,b7_prime,b7,self.one); W += self._term_sinh(I6,k16,b8_prime,b8,self.one)
        return W

In [6]:
@tf.function
def get_invariants_tf(lambda1, lambda2, lambda3):
    min_lambda_val = tf.constant(1e-6, dtype=tf.float64)
    lambda1 = tf.maximum(lambda1, min_lambda_val)
    lambda2 = tf.maximum(lambda2, min_lambda_val)
    lambda3 = tf.maximum(lambda3, min_lambda_val)
    l1s = tf.pow(lambda1, 2.0); l2s = tf.pow(lambda2, 2.0); l3s = tf.pow(lambda3, 2.0)
    I1 = l1s + l2s + l3s
    I2 = tf.pow(lambda1 * lambda2, 2.0) + tf.pow(lambda2 * lambda3, 2.0) + tf.pow(lambda3 * lambda1, 2.0)
    I4 = l1s
    # I6 has an alternative definition depending on fiber orientation. Assuming a_0 = [1,0,0]
    I6 = 1/l1s
    return I1, I2, I4, I6


In [7]:

@tf.function
def predict_uniaxial_p11(lambda1, model):
    l1 = lambda1
    with tf.GradientTape(persistent=True) as tape:
        l1_t = tf.identity(l1)
        # Incompressible material: lambda1 * lambda2 * lambda3 = 1
        l2_t = tf.pow(l1_t, -0.5)
        l3_t = tf.pow(l1_t, -0.5)
        tape.watch([l1_t, l2_t, l3_t])
        I1, I2, I4, I6 = get_invariants_tf(l1_t, l2_t, l3_t)
        W = model(I1, I2, I4, I6)

    dWdl1 = tape.gradient(W, l1_t)
    dWdl2 = tape.gradient(W, l2_t)
    dWdl3 = tape.gradient(W, l3_t)
    del tape

    # Cauchy stress T = (2/J) * [ (dW/dI1 + I1*dW/dI2)*B - (dW/dI2)*B^2 ... ] * F^T
    # For uniaxial, P11 = 2 * (l1 - 1/l1^2) * (dW/dI1 + 1/l1 * dW/dI2) + 2 * (l1-1/l1^2)*dW/dI4
    # A more direct approach using principal stresses for incompressible materials:
    # P_ii = dW/d_lambda_i - p / lambda_i
    # For uniaxial test, P22 = P33 = 0. So p = lambda2 * dW/d_lambda2 = lambda3 * dW/d_lambda3
    p = l2_t * dWdl2
    P11 = dWdl1 - p / l1_t
    return P11

In [8]:
@tf.function
def get_biaxial_stresses(l1, l2, model):
    with tf.GradientTape(persistent=True) as tape:
        l1_t = tf.identity(l1)
        l2_t = tf.identity(l2)
        # Incompressible material: lambda1 * lambda2 * lambda3 = 1
        l3_t = 1.0 / (l1_t * l2_t)
        tape.watch([l1_t, l2_t, l3_t])
        I1, I2, I4, I6 = get_invariants_tf(l1_t, l2_t, l3_t)
        W = model(I1, I2, I4, I6)

    dWdl1 = tape.gradient(W, l1_t)
    dWdl2 = tape.gradient(W, l2_t)
    dWdl3 = tape.gradient(W, l3_t)
    del tape

    # For planar tension, P33 = 0. So p = lambda3 * dW/d_lambda3
    p = l3_t * dWdl3
    P11 = dWdl1 - p / l1_t
    P22 = dWdl2 - p / l2_t
    return P11, P22


In [9]:
@tf.function
def predict_biaxial_p22(lambda2, model):
    l1_min = tf.ones_like(lambda2, dtype=tf.float64) * 0.5
    l1_max = tf.ones_like(lambda2, dtype=tf.float64) * 1.5

    # Use bisection method to find lambda1 such that P11=0
    for _ in range(25): # Increased iterations for better precision
        l1_mid = (l1_min + l1_max) / 2.0
        p11_mid, _ = get_biaxial_stresses(l1_mid, lambda2, model)
        p11_min, _ = get_biaxial_stresses(l1_min, lambda2, model)

        is_same_sign = tf.sign(p11_mid) == tf.sign(p11_min)
        l1_min = tf.where(is_same_sign, l1_mid, l1_min)
        l1_max = tf.where(is_same_sign, l1_max, l1_mid)

    final_l1 = tf.stop_gradient((l1_min + l1_max) / 2.0)
    _, P22_final = get_biaxial_stresses(final_l1, lambda2, model)
    return P22_final


In [10]:
# --- Experimental Data ---
exp_data_raw_uniaxial_cnf = np.array([
    [1.0000, 0], [1.0708, 0.3840], [1.2017, 0.8987], [1.3125, 1.1814], [1.4000, 1.4093],
    [1.5125, 1.6456], [1.6017, 1.8608], [1.7125, 2.1055], [1.8008, 2.3122], [1.8883, 2.5570],
    [1.9767, 2.7848], [2.0883, 3.1519], [2.1992, 3.5274], [2.2867, 3.8354], [2.3975, 4.2532],
    [2.4383, 4.4304], [2.4858, 4.5949]
])
exp_data_raw_biaxial_cnf = np.array([
    [1.0000, 0], [1.3208, 1.0506], [1.4017, 1.2068], [1.5092, 1.3840], [1.5983, 1.5401],
    [1.7017, 1.6835], [1.7842, 1.7848], [1.8967, 1.9662], [1.9792, 2.1181], [2.0858, 2.2911],
    [2.1708, 2.4599], [2.2783, 2.6962], [2.3825, 2.9409], [2.4225, 3.0549], [2.4867, 3.2236]
])
uniaxial_l1, uniaxial_p11 = [tf.constant(c, dtype=tf.float64) for c in exp_data_raw_uniaxial_cnf.T]
biaxial_l2, biaxial_p22 = [tf.constant(c, dtype=tf.float64) for c in exp_data_raw_biaxial_cnf.T]

In [ ]:
# --- Training with CONSTRAINED & IMPROVED LOSS FUNCTION ---

model = StrainEnergyANN_Layered_TF()

initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=2000, decay_rate=0.9, staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

loss_weight_biaxial = tf.constant(1.0, dtype=tf.float64)
# --- MODIFICATION: ADD WEIGHT FOR THE NEW ZERO-STRESS PENALTY ---
zero_stress_weight = tf.constant(100.0, dtype=tf.float64)

print(f"Biaxial Loss Weight: {loss_weight_biaxial.numpy():.2f}")
print(f"Zero-Stress Penalty Weight: {zero_stress_weight.numpy():.2f}")

In [ ]:
# --- Visualization ---

# Generate predictions for plotting
l1_plot = np.linspace(1.0, exp_data_raw_uniaxial_cnf[:, 0].max(), 100)
p11_pred_plot = predict_uniaxial_p11(tf.constant(l1_plot, dtype=tf.float64), model).numpy()

l2_plot = np.linspace(1.0, exp_data_raw_biaxial_cnf[:, 0].max(), 100)
p22_pred_plot = predict_biaxial_p22(tf.constant(l2_plot, dtype=tf.float64), model).numpy()

# Create plots
plt.style.use('seaborn-v0_8-whitegrid')
fig, axes = plt.subplots(1, 3, figsize=(21, 6))

# Uniaxial Plot
axes[0].plot(exp_data_raw_uniaxial_cnf[:, 0], exp_data_raw_uniaxial_cnf[:, 1], 'ro', label='Experimental Data')
axes[0].plot(l1_plot, p11_pred_plot, 'b-', lw=2, label='Model Prediction (P11)')
axes[0].set_xlabel('Stretch $\\lambda_1$', fontsize=12)
axes[0].set_ylabel('Nominal Stress $P_{11}$ (MPa)', fontsize=12)
axes[0].set_title('Task 1: Uniaxial Tension Fit', fontsize=14)
axes[0].legend()
axes[0].set_ylim(bottom=0)
axes[0].set_xlim(left=1)

# Biaxial Plot
axes[1].plot(exp_data_raw_biaxial_cnf[:, 0], exp_data_raw_biaxial_cnf[:, 1], 'go', label='Experimental Data')
axes[1].plot(l2_plot, p22_pred_plot, 'b-', lw=2, label='Model Prediction (P22)')
axes[1].set_xlabel('Stretch $\\lambda_2$', fontsize=12)
axes[1].set_ylabel('Nominal Stress $P_{22}$ (MPa)', fontsize=12)
axes[1].set_title('Task 2: Biaxial (Planar) Tension Fit', fontsize=14)
axes[1].legend()
axes[1].set_ylim(bottom=0)
axes[1].set_xlim(left=1)

# Loss History Plot
ax2 = axes[2]
ax2.plot(loss_history, label='Total Loss', color='darkblue')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Total Loss (MSRE + Penalty)', fontsize=12, color='darkblue')
ax2.set_title('Training History', fontsize=14)
ax2.set_yscale('log')
ax2.tick_params(axis='y', labelcolor='darkblue')
ax2.grid(True, which="both", ls="--")

# Learning Rate Plot on the same axes
ax3 = ax2.twinx()
ax3.plot(learning_rate_history, label='Learning Rate', color='red', linestyle='--')
ax3.set_ylabel('Learning Rate', fontsize=12, color='red')
ax3.tick_params(axis='y', labelcolor='red')

fig.tight_layout()
plt.show()

# --- Final Check ---
# Final check to confirm stresses are very close to zero at lambda=1
p11_final_check = predict_uniaxial_p11(tf.constant([1.0], dtype=tf.float64), model)
p22_final_check = predict_biaxial_p22(tf.constant([1.0], dtype=tf.float64), model)
print("\nFinal check:")
print(f"P11 at lambda1=1: {p11_final_check.numpy()[0]:.6e}")
print(f"P22 at lambda2=1: {p22_final_check.numpy()[0]:.6e}")